In [ ]:
#
# Script meant to test the ability to show brief "help" animations in the interactive graph panel
# ... animations still showing incorrect behavior
#
import polars as pl
import networkx as nx
import rtsvg
rt = rtsvg.RACETrack()

_base_network_ = '0'
_base_dir_     = '../../data/stanford/facebook/'
_layout_file_  = _base_dir_ + _base_network_ + '.layout.parquet'
_edges_ = open(_base_dir_ + _base_network_ + '.edges', 'rt').read()
_lu_ = {'fm':[], 'to':[]}
for _edge_ in _edges_.split('\n'):
    if _edge_ == '': continue
    _lu_['fm'].append(_edge_.split(' ')[0])
    _lu_['to'].append(_edge_.split(' ')[1])
df        = pl.DataFrame(_lu_)
_relates_ = [('fm','to')]
g         = rt.createNetworkXGraph(df, _relates_)

# Provides colors based on community
_node_colors_ = {}
community_i   = 0
for _community_ in nx.community.louvain_communities(g):
    community_i += 1
    for _node_ in _community_: _node_colors_[_node_] = rt.co_mgr.getColor(community_i)

#_rtg_ = rt.interactiveGraphPanel(df, {'relationships':_relates_, 'node_color':_node_colors_}, w=800, h=500)
#_rtg_

In [ ]:
#_rtg_.setAnimation('<rect x="10" y="10" width="10" height="10" fill="blue"/>')

In [ ]:
#_rtg_.setAnimation('<text x="5" y="15" fill="black"> popStack </text>')

In [ ]:
from rtsvg.polars_force_directed_layout import PolarsForceDirectedLayout
_pfdl_ = PolarsForceDirectedLayout(g)

In [ ]:
def stress(self, anim_i=0, k=0):
    df_pos  = self.df_results[anim_i]
    df_dist = self.df_dist[anim_i]
    _df_    = df_pos.join(df_pos, how='cross') \
                    .filter(pl.col('node') != pl.col('node_right')) \
                    .join(df_dist, left_on=['node', 'node_right'], right_on=['fm','to']) \
                    .with_columns(((pl.col('x') - pl.col('x_right'))**2 + (pl.col('y') - pl.col('y_right'))**2).sqrt().alias('d')) \
                    .with_columns((pl.col('t')**(2-k)).alias('__prod_1__'),
                                  ((pl.col('d') - pl.col('t'))**2 / pl.col('t')**k).alias('__prod_2__'))
    return (1.0 / _df_['__prod_1__'].sum()) * _df_['__prod_2__'].sum()

stress(_pfdl_, k=0), stress(_pfdl_, k=1), stress(_pfdl_, k=2)

In [ ]:
_pfdl_.stress(k=0), _pfdl_.stress(k=1), _pfdl_.stress(k=2)